# Law School Debt Calculator - Data Retrieval, Cleaning, & Upload

In [1]:
import os
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

## Establish DB Connection

In [40]:
def get_db_conn(local=True):
    if local:
        conn_string = os.environ["PG_LOCAL"]
    else:
        conn_string = os.environ["PG_HEROKU"]
    
    return create_engine(conn_string)

db_engine = get_db_conn(local=True)

## Load & Clean ABA Data

### ABA Disclosures Data
- Grants & Scholarships
- Tuition & Fees, CoL
- Employment Rates

These are currently pulled manually from the [ABA Disclosures Site](http://www.abarequireddisclosures.org/Disclosure509.aspx) in Excel form, and loaded from the /data/ directory

In [3]:
file_list = os.listdir("./data/aba/")

file_list

['cost_of_attendance_2020.xlsx',
 'employment_summary_2020.xlsx',
 'basics_2020.xlsx',
 'grants_scholarships_2020.xlsx']

In [4]:
df_dict = {i[:-5]: pd.read_excel("./data/aba/"+i, engine="openpyxl") for i in file_list}

pd.set_option("max_columns", 100)

for k, v in df_dict.items():
    print(f"\'{k}\' - Shape: {v.shape}")

'cost_of_attendance_2020' - Shape: (197, 31)
'employment_summary_2020' - Shape: (198, 159)
'basics_2020' - Shape: (197, 9)
'grants_scholarships_2020' - Shape: (197, 43)


### Basics

In [5]:
basics_load_df = df_dict["basics_2020"]

basics_load_df.head()

,School List,Type of School,Application Deadline,Application Fee,Fin Aid Deadline,Term,Months students begin,Months degrees conferred,# of Credit Hours for JD
0,"AKRON, UNIVERSITY OF",Public,03/31/2021,$0,"3/31/2021, 11/30/2021",Semester,"August, January, May","January, May",88
1,"ALABAMA, UNIVERSITY OF",Public,NaN,$40,NaN,Semester,August,May/ December,90
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,Private,NaN,NaN,NaN,Semester,"January, August","December, May, July",87
3,AMERICAN UNIVERSITY,Private,3/1,$70.00,3/1,Semester,August,"May, August, December",86
4,APPALACHIAN SCHOOL OF LAW,Private,August 1,$0,August 1,Semester,August and January,"May, August, and December",92


In [6]:
basics_cols = {
    "School List":"school",
    "Type of School":"school_type",
    "# of Credit Hours for JD":"grad_credit_hours",
}

basics_df = basics_load_df[basics_cols.keys()].rename(columns=basics_cols)

basics_df.head()

,school,school_type,grad_credit_hours
0,"AKRON, UNIVERSITY OF",Public,88
1,"ALABAMA, UNIVERSITY OF",Public,90
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,Private,87
3,AMERICAN UNIVERSITY,Private,86
4,APPALACHIAN SCHOOL OF LAW,Private,92


### Grants and Scholarships

In [7]:
gns_load_df = df_dict["grants_scholarships_2020"]

gns_load_df.head()

,SchoolName,Total # total students #,Total # total of Students %,Total # of students FT #,Total # of students FT %,Total # of students PT #,Total # of students PT %,Total # receiving grants total #,Total # receiving grants total %,Total # receiving grants FT #,Total # receiving grants FT %,Total # receiving grants PT #,Total # receiving grants PT %,Less than half tuition total #,Less than half tuition total %,Less than half tuition FT #,Less than half tuition FT %,Less than half tuition PT #,Less than half tuition PT %,Half to full tuition total #,Half to full tuition total %,half to full tuition FT #,half to full tuition FT %,half to full tuition PT #,half to full tuition PT %,Full tuition total #,Full tuition total %,Full tuition FT #,Full tuition FT %,Full tuition PT #,Full tuition PT %,More than full tuition total #,More than full tuition total %,More than full tuition FT #,More than full tuition FT %,More than full tuition PT #,More than full tuition PT %,FT 75th percentile grant amount,FT 50th percentile grant amount,FT 25th percentile grant amount,PT 75th percentile grant amount,PT 50th percentile grant amount,PT 25th percentile grant amount
0,"AKRON, UNIVERSITY OF",463,100,351,76,112,24,345,75,284,81,61,54,259,56,219,62,40,36,81,17,62,18,19,17,3,1,3,1,0,0,2,0,0,0,2,2,"4,000","7,000","12,000","3,000","5,500","9,000"
1,"ALABAMA, UNIVERSITY OF",379,100,379,100,0,0,326,86,326,86,0,0,86,23,86,23,0,0,165,44,165,44,0,0,16,4,16,4,0,0,59,16,59,16,0,0,"35,000","24,000","15,000",0,0,0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,493,100,493,100,0,0,458,93,443,90,15,0,173,35,166,34,7,0,236,48,229,46,7,0,48,10,47,10,1,0,1,0,1,0,0,0,"40,000","31,000","18,000","30,000","22,500","13,500"
3,AMERICAN UNIVERSITY,1242,100,1025,83,217,17,917,74,773,75,144,66,220,18,200,20,20,9,618,50,502,49,116,53,77,6,69,7,8,4,2,0,2,0,0,0,"35,000","30,000","25,000","30,000","30,000","25,000"
4,APPALACHIAN SCHOOL OF LAW,171,100,171,100,0,0,102,60,102,60,0,0,62,36,62,36,0,0,23,13,23,13,0,0,6,4,6,4,0,0,11,6,11,6,0,0,"17,500","9,475","5,000",0,0,0


In [8]:
gns_cols = {
    "SchoolName":"school",
    "Total # receiving grants total %":"percent_get_grant",
    "FT 50th percentile grant amount":"median_grant",
}

gns_df  = gns_load_df[gns_cols.keys()].rename(columns=gns_cols)

gns_df.head()

,school,percent_get_grant,median_grant
0,"AKRON, UNIVERSITY OF",75,"7,000"
1,"ALABAMA, UNIVERSITY OF",86,"24,000"
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,93,"31,000"
3,AMERICAN UNIVERSITY,74,"30,000"
4,APPALACHIAN SCHOOL OF LAW,60,"9,475"


### Cost of Attendance

In [9]:
coa_load_df = df_dict["cost_of_attendance_2020"]

coa_load_df[25:30]

,school list,Full Time Resident Semester,FTRS Annual Fees,Full Time Non resident Semester,FTNRS Annual Fees,Part Time Resident Semester,PTRS Annual Fees,Part Time Non resident Semester,PTNRS Annual Fees,Full Time Resident Credit,FTRSPerCredit Annual Fees,Full Time Non resident Credit,FTNRSPerCredit Annual Fees,Part Time Resident Credit,PTRSPerCredit Annual Fees,Part Time Non resident Credit,PTNRSPerCredit Annual Fees,Other Semester,Other Semester Annual Fees,Other Credit,Other Per Credit Annual Fees,Tuition Guarantee Program,Living On Campus,Living Off Campus,Living At Home,19-20 Entering scholarships,19-20 Reduced/Eliminated Scholarships,18-19 Entering scholarships,18-19 Reduced/Eliminated Scholarships,17-18 Entering scholarships,17-18 Reduced/Eliminated Scholarships
25,CAMPBELL UNIVERSITY,"22,475",830,"22,475",830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,No,"26,560","26,560","17,460",0,0,0,0,0,0
26,CAPITAL UNIVERSITY,"18,676",220,"18,676",220,"14,674",110,"14,674",110,"1,334",220,"1,334",220,"1,334",110,"1,334",110,0,0,0,0,No,"18,504","18,504","18,504",158,0,118,32,117,29
27,CARDOZO SCHOOL OF LAW,"31,950",0,"31,950",0,"21,300",0,"21,300",0,0,0,0,0,0,0,0,0,0,0,"2,849",0,No,"29,189","29,189","29,189",0,0,0,0,0,0
28,CASE WESTERN RESERVE UNIVERSITY,"27,550",178,"27,550",178,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,No,"26,230","26,230","26,230",0,0,0,0,0,0
29,CATHOLIC UNIVERSITY OF AMERICA,"26,475","1,280","26,475","1,280",0,0,0,0,0,0,0,0,"1,825","1,020","1,825","1,020",0,0,0,0,No,"26,192","31,620","16,382",39,0,22,3,18,3


In [10]:
coa_cols = {
    "school list":"school",
    "Full Time Resident Semester":"tuition_semester_res",
    "Full Time Non resident Semester":"tuition_semester_nonres",
    "FTRS Annual Fees":"fees_semester_res",
    "FTNRS Annual Fees":"fees_semester_nonres",
    "Full Time Resident Credit":"tuition_credit_res",
    "Full Time Non resident Credit":"tuition_credit_nonres",
    "FTRSPerCredit Annual Fees":"fees_credit_res",
    "FTNRSPerCredit Annual Fees":"fees_credit_nonres",
    "Living On Campus":"col_on_campus",
    "Living Off Campus":"col_off_campus",
    "Living At Home":"col_at_home"
}

coa_df  = coa_load_df[coa_cols.keys()].rename(columns=coa_cols)

coa_df

,school,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home
0,"AKRON, UNIVERSITY OF","12,107","12,157","1,419","1,469",712,712,"1,419","1,469","22,404","22,404","7,038"
1,"ALABAMA, UNIVERSITY OF","11,805","21,530",310,310,0,0,0,0,"19,010","19,010","19,010"
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,"26,053","26,053",135,135,0,0,0,0,0,"16,640","7,690"
3,AMERICAN UNIVERSITY,"26,018","26,018",905,905,0,0,0,0,"23,872","23,872","23,872"
4,APPALACHIAN SCHOOL OF LAW,"17,500","17,500",500,500,0,0,0,0,"20,100","20,100","20,100"
...,...,...,...,...,...,...,...,...,...,...,...,...
192,WILLAMETTE UNIVERSITY,"23,565","23,265",195,195,0,0,0,0,"20,282","20,282","20,282"
193,WILLIAM AND MARY LAW SCHOOL,"14,362","18,576","6,276","6,848",0,0,0,0,"21,152","21,152","21,152"
194,"WISCONSIN, UNIVERSITY OF","13,118","22,466","1,468","1,468","1,093","1,872","1,468","1,468","23,104","23,104","11,912"
195,"WYOMING, UNIVERSITY OF",0,0,0,0,483,"1,030","1,441","1,441","17,292","17,292","17,292"


### Employment

In [11]:
emp_load_df = df_dict["employment_summary_2020"]

pd.set_option("max_columns", 100)
emp_load_df.head()

,SchoolName,Employed_BarPassageRequiredFTLT,Employed_BarPassageRequiredFTST,Employed_BarPassageRequiredPTLT,Employed_BarPassageRequiredPTST,Employed_BarPassageRequiredNumber,Employed_JDAdvantageFTLT,Employed_JDAdvantageFTST,Employed_JDAdvantagePTLT,Employed_JDAdvantagePTLST,Employed_JDAdvantageNumber,Employed_ProfessionPositionFTLT,Employed_ProfessionPositionFTST,Employed_ProfessionPositionPTLT,Employed_ProfessionPositionPTST,Employed_ProfessionPositionNumber,Employed_NonProfessionPositionFTLT,Employed_NonProfessionPositionFTST,Employed_NonProfessionPositionPTLT,Employed_NonProfessionPositionPTST,Employed_NonProfessionPositionNumber,Employed_LawSchoolFTLT,Employed_LawSchoolFTST,Employed_LawSchoolPTLT,Employed_LawSchoolPTST,Employed_LawSchoolNumber,Employed_UndeterminableFTLT,Employed_UndeterminableFTST,Employed_UndeterminablePTLT,Employed_UndeterminablePTST,Employed_UndeterminableNumber,PursuingGraduateDegreeNumber,EmployedStartDateDeferredNumber,UnEmployedNotSeekingNumber,UnEmployedSeekingNumber,EmploymentStatusUnknownNumber,Total_GraduatesNumber,Funded_BarPassgeFullTimeLongTerm,Funded_BarPassgeFullTimeShortTerm,Funded_BarPassgePartTimeLongTerm,Funded_BarPassgePartTimeShortTerm,Funded_BarPassgeTotalEmployed,Funded_JDAdvantageFullTimeLongTerm,Funded_JDAdvantageFullTimeShortTerm,Funded_JDAdvantagePartTimeLongTerm,Funded_JDAdvantagePartTimeShortTerm,Funded_JDAdvantageTotalEmployed,Funded_ProfessionFullTimeLongTerm,Funded_ProfessionFullTimeShortTerm,Funded_ProfessionPartTimeLongTerm,...,BusinessIndustry_PTLT,BusinessIndustry_PTST,BusinessIndustry,Government_FTLT,Government_FTST,Government_PTLT,Government_PTST,Government,PublicInterest_FTLT,PublicInterest_FTST,PublicInterest_PTLT,PublicInterest_PTST,PublicInterest,Clerkships_Federal_FTLT,Clerkships_Federal_FTST,Clerkships_Federal_PTLT,Clerkships_Federal_PTST,Clerkships_Federal,Clerkships_StateLocal_FTLT,Clerkships_StateLocal_FTST,Clerkships_StateLocal_PTLT,Clerkships_StateLocal_PTST,Clerkships_StateLocal,Clerkships_Other_FTLT,Clerkships_Other_FTST,Clerkships_Other_PTLT,Clerkships_Other_PTST,Clerkships_Other,Education_FTLT,Education_FTST,Education_PTLT,Education_PTST,Education,EmployerTypeUnknown_FTLT,EmployerTypeUnknown_FTST,EmployerTypeUnknown_PTLT,EmployerTypeUnknown_PTST,EmployerTypeUnknown,Total_FTLT,Total_FTST,Total_PTLT,Total_PTST,Total,FirstLargestEmployment,FirstLargestEmploymentNumber,SecondLargestEmployment,SecondLargestEmploymentNumber,ThirdLargestEmployment,ThirdLargestEmploymentNumber,EmployedInForeignCountries
0,"AKRON, UNIVERSITY OF",82,1,2,1,86,24,0,3,0,27,2,0,2,0,4,1,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,129,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,34,14,0,1,1,16,3,0,0,0,3,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,109,1,9,1,120,Ohio,103,Pennsylvania,6,New York,2,1
1,"ALABAMA, UNIVERSITY OF",107,1,0,1,109,12,0,0,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,7,0,133,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,16,18,0,0,0,18,2,0,0,0,2,11,0,0,0,11,4,0,0,0,4,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,119,1,0,2,122,Alabama,60,Texas,10,Georgia,8,0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,99,0,3,0,102,12,3,3,1,19,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,11,0,136,1,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,8,26,3,1,0,30,6,0,0,0,6,3,0,0,0,3,9,0,0,0,9,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,113,3,6,1,123,New York,110,Massachusetts,3,District of Columbia,2,0
3,AMERICAN UNIVERSITY,252,3,4,2,261,49,6,1,5,61,8,2,2,0,12,1,0,0,2,3,2,0,0,0,2,0,0,0,0,0,4,4,2,49,5,403,2,0,0,0,2,0,0,0,0,0,0,0,0,...,1,4,43,71,4,1,2,78,36,0,1,1,38,11,0,1,0,12,37,2,0,0,39,0,0,0,0,0,3,0,0,1,4,0,0,0,0,0,312,11,7,9,339,District of Columbia,147,Virginia,46,Maryland,44,9
4,APPALACHIAN SCHOOL OF LAW,13,0,0,0,13,3,1,0,0,4,0,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,4,8,0,31,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,0,0,0,2,2,0,0,0,2,0,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,1,1,0,19,Virginia,8,West Virginia,4,Kentucky,3,0


In [12]:
emp_cols = {
    "SchoolName":"school",
    "UnEmployedNotSeekingNumber":"unemployed_not_seeking",
    "UnEmployedSeekingNumber":"unemployed_seeking",
    "Total_GraduatesNumber":"total_graduates"
}

emp_df  = emp_load_df[emp_cols.keys()].rename(columns=emp_cols)

emp_df

,school,unemployed_not_seeking,unemployed_seeking,total_graduates
0,"AKRON, UNIVERSITY OF",0,8,129
1,"ALABAMA, UNIVERSITY OF",1,7,133
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,1,11,136
3,AMERICAN UNIVERSITY,2,49,403
4,APPALACHIAN SCHOOL OF LAW,4,8,31
...,...,...,...,...
193,WILLAMETTE UNIVERSITY,1,9,87
194,WILLIAM AND MARY LAW SCHOOL,0,10,230
195,"WISCONSIN, UNIVERSITY OF",0,3,155
196,"WYOMING, UNIVERSITY OF",2,3,70


### Join ABA Data

In [13]:
# Join all DFs from ABA data
aba_df = (gns_df.join(coa_df.set_index("school"), on="school")
                .join(emp_df.set_index("school"), on="school")
                .join(basics_df.set_index("school"), on="school"))

# Fix credit hours column so can be converted to numeric type
aba_df.loc[aba_df["grad_credit_hours"]=="84 or 88", "grad_credit_hours"] = "86"

# Change columns with commas to numeric
wrong_type_cols = [
    "median_grant",
    "tuition_semester_res",
    "tuition_semester_nonres",
    "fees_semester_res",
    "fees_semester_nonres",
    "tuition_credit_res",
    "tuition_credit_nonres",
    "fees_credit_res",
    "fees_credit_nonres",
    "col_on_campus",
    "col_off_campus",
    "col_at_home",
    "grad_credit_hours"
]

aba_df[wrong_type_cols] = aba_df[wrong_type_cols].replace(",", "", regex=True).astype(float)

aba_df.head()

,school,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours
0,"AKRON, UNIVERSITY OF",75,7000.0,12107.0,12157.0,1419.0,1469.0,712.0,712.0,1419.0,1469.0,22404.0,22404.0,7038.0,0,8,129,Public,88.0
1,"ALABAMA, UNIVERSITY OF",86,24000.0,11805.0,21530.0,310.0,310.0,0.0,0.0,0.0,0.0,19010.0,19010.0,19010.0,1,7,133,Public,90.0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,93,31000.0,26053.0,26053.0,135.0,135.0,0.0,0.0,0.0,0.0,0.0,16640.0,7690.0,1,11,136,Private,87.0
3,AMERICAN UNIVERSITY,74,30000.0,26018.0,26018.0,905.0,905.0,0.0,0.0,0.0,0.0,23872.0,23872.0,23872.0,2,49,403,Private,86.0
4,APPALACHIAN SCHOOL OF LAW,60,9475.0,17500.0,17500.0,500.0,500.0,0.0,0.0,0.0,0.0,20100.0,20100.0,20100.0,4,8,31,Private,92.0


### Clean ABA Data

In [14]:
# Fill blanks in residential and non-residential tuitions with the other
aba_df.loc[aba_df["tuition_semester_res"]==0, "tuition_semester_res"] = aba_df["tuition_semester_nonres"]
aba_df.loc[aba_df["tuition_semester_nonres"]==0, "tuition_semester_nonres"] = aba_df["tuition_semester_res"]

In [15]:
# Fill blanks in semester-based residential and non-residential tuitions with per-credit tuition
aba_df.loc[aba_df["tuition_semester_res"]==0, "tuition_semester_res"] = aba_df["tuition_credit_res"]*aba_df["grad_credit_hours"]/6
aba_df.loc[aba_df["tuition_semester_nonres"]==0, "tuition_semester_nonres"] = aba_df["tuition_credit_nonres"]*aba_df["grad_credit_hours"]/6

In [16]:
# Fill blanks in residential and non-residential fees from credit fees section
aba_df.loc[aba_df["fees_semester_res"]==0, "fees_semester_res"] = aba_df["fees_credit_res"]
aba_df.loc[aba_df["fees_semester_nonres"]==0, "fees_semester_nonres"] = aba_df["fees_credit_nonres"]

In [17]:
# Fill blanks in residential and non-residential tuitions from credit calculations
aba_df.loc[aba_df["tuition_semester_res"]==0, "tuition_semester_res"] = aba_df["tuition_semester_nonres"]
aba_df.loc[aba_df["tuition_semester_nonres"]==0, "tuition_semester_nonres"] = aba_df["tuition_semester_res"]

In [18]:
# This should be empty
aba_df[(aba_df["tuition_semester_res"]==0)|(aba_df["tuition_semester_nonres"]==0)]

,school,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours


## Pull Salary Statistics
Scrape from [publiclegal](https://www.ilrg.com/rankings/law/median/1/desc/MSPrivate)

In [19]:
from bs4 import BeautifulSoup
from html_table_extractor.extractor import Extractor
import requests

In [20]:
def extract_table(url, table_number=0):
    response = requests.get(url)
    content = response.content

    html = BeautifulSoup(content, 'html.parser')
    table_list = html.find_all("table")

    table = table_list[table_number]
    
    extractor = Extractor(table)
    extractor.parse()
    
    table_data = extractor.return_list()
    
    return pd.DataFrame(table_data[1:], columns=table_data[0])

In [21]:
urls = [f"https://www.ilrg.com/rankings/law/median/{i}/desc/MSPrivate" for i in range(1,5)]

salary_df = pd.concat([extract_table(url, table_number=1) for url in urls]).reset_index(drop=True)

# Remove weird line returns
salary_df = salary_df.replace("\n", "", regex=True)

salary_df.columns = salary_df.columns.to_series().replace("\n", "", regex=True)

# Change salary columns to floats
clean_salary = lambda df: df.replace("N/A", np.nan).replace("\$|,", "", regex=True).astype(float)
salary_df["Median Salary Private"] = salary_df["Median Salary Private"].pipe(clean_salary)
salary_df["Median Salary Public"] = salary_df["Median Salary Public"].pipe(clean_salary)

salary_df

,No.,Law School,State,Median Salary Private,Median Salary Public
0,1,Vanderbilt University,TN,180000.0,65287.0
1,1,Stanford University,CA,180000.0,65228.0
2,1,Cornell University,NY,180000.0,64228.0
3,1,U. of Chicago,IL,180000.0,63000.0
4,1,Yale University,CT,180000.0,62591.0
...,...,...,...,...,...
195,196,U. of New Hampshire,NH,NaN,NaN
196,197,U. of North Dakota,ND,NaN,NaN
197,198,Valparaiso University,IN,NaN,NaN
198,199,Western Michigan (Cooley),MI,NaN,NaN


## Match Datasets on Law School Name

In [22]:
aba_df["school_clean"] = aba_df["school"].replace(r"^(.*), (.*?)$", r"\2 \1", regex=True)

In [23]:
salary_df["school_clean"] = salary_df["Law School"].replace(r"U\.", r"University", regex=True).str.upper()

In [24]:
from jellyfish import levenshtein_distance as ld
from jellyfish import jaro_winkler_similarity as jws

In [25]:
salary_df["school_clean"].sort_values().head()

89     ALBANY LAW SCHOOL UNION UNIVERSITY
40                    AMERICAN UNIVERSITY
185             APPALACHIAN SCHOOL OF LAW
63               ARIZONA STATE UNIVERSITY
186             ARIZONA SUMMIT LAW SCHOOL
Name: school_clean, dtype: object

In [26]:
regex = r"university|college| of(?: |$)| at |,|law|school|U\.|Univ\."

aba_df["school_root"] = aba_df["school_clean"].str.replace(regex, "", regex=True, flags=re.I)
salary_df["school_root"] = salary_df["school_clean"].str.replace(regex, "", regex=True, flags=re.I)

In [27]:
def school_match(school):
    return aba_df.loc[aba_df["school_clean"].apply(lambda x: jws(x, school)).idxmax(), "school"]

salary_df["school_match"] = salary_df["school_clean"].apply(school_match)

salary_df[["Law School", "school_match"]].head()

,Law School,school_match
0,Vanderbilt University,VANDERBILT UNIVERSITY
1,Stanford University,STANFORD UNIVERSITY
2,Cornell University,CORNELL UNIVERSITY
3,U. of Chicago,"CHICAGO, UNIVERSITY OF"
4,Yale University,YALE UNIVERSITY


In [28]:
def school_root_match(school):
    return aba_df.loc[aba_df["school_root"].apply(lambda x: jws(x, school)).idxmax(), "school"]

salary_df["school_match"] = salary_df["school_root"].apply(school_root_match)

In [29]:
filters = (
    (aba_df["school"].str.contains("concordia", case=False, regex=True))
)

aba_df.loc[filters]

,school,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours,school_clean,school_root


In [30]:
manual_map = {
    "Loyola Law School":"LOYOLA MARYMOUNT UNIVERSITY-LOS ANGELES",
    "U. of N. Carolina-Chap. Hill":"NORTH CAROLINA, UNIVERSITY OF",
    "U. of St. Thomas":"ST. THOMAS, UNIVERSITY OF (MINNESOTA)",
    "Boston University":"BOSTON UNIVERSITY",
    "CUNY-Queens College":"CITY UNIVERSITY OF NEW YORK",
    "St. Louis University":"SAINT LOUIS UNIVERSITY",
    "Charlotte School of Law":"",
    "Hamline University":"",
    "U. of the Pacific (McGeorge)":"",
    "Arizona Summit Law School":"",
    "Concordia University":"",
    "Thomas Jefferson":"",
    "Valparaiso University":"",
    "Whittier Law School":"",
}

for school in manual_map.keys():
    salary_df.loc[salary_df["Law School"]==school, "school_match"] = manual_map.get(school)
    
# pd.set_option("max_rows", 10)
    
salary_df[["Law School", "school_match"]].sort_values("Law School")

,Law School,school_match
89,Albany Law School Union U.,ALBANY LAW SCHOOL OF UNION UNIVERSITY
40,American University,AMERICAN UNIVERSITY
185,Appalachian School of Law,APPALACHIAN SCHOOL OF LAW
63,Arizona State University,ARIZONA STATE UNIVERSITY
186,Arizona Summit Law School,
...,...,...
199,Whittier Law School,
110,Widener (Commonwealth),WIDENER-COMMONWEALTH
150,Widener (Delaware),WIDENER UNIVERSITY-DELAWARE
143,Willamette University,WILLAMETTE UNIVERSITY


In [31]:
salary_df["match_idx"] = salary_df["school_match"].map(aba_df.reset_index().set_index("school")["index"])

In [32]:
final_df = aba_df.join(salary_df.set_index("match_idx").drop(["school_clean", "school_root"], axis=1))

final_df.head()

,school,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours,school_clean,school_root,No.,Law School,State,Median Salary Private,Median Salary Public,school_match
0,"AKRON, UNIVERSITY OF",75,7000.0,12107.0,12157.0,1419.0,1469.0,712.0,712.0,1419.0,1469.0,22404.0,22404.0,7038.0,0,8,129,Public,88.0,UNIVERSITY OF AKRON,AKRON,142,U. of Akron,OH,60000.0,42198.0,"AKRON, UNIVERSITY OF"
1,"ALABAMA, UNIVERSITY OF",86,24000.0,11805.0,21530.0,310.0,310.0,0.0,0.0,0.0,0.0,19010.0,19010.0,19010.0,1,7,133,Public,90.0,UNIVERSITY OF ALABAMA,ALABAMA,95,U. of Alabama,AL,70000.0,50000.0,"ALABAMA, UNIVERSITY OF"
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,93,31000.0,26053.0,26053.0,135.0,135.0,0.0,0.0,0.0,0.0,0.0,16640.0,7690.0,1,11,136,Private,87.0,ALBANY LAW SCHOOL OF UNION UNIVERSITY,ALBANY UNION,90,Albany Law School Union U.,NY,73000.0,59000.0,ALBANY LAW SCHOOL OF UNION UNIVERSITY
3,AMERICAN UNIVERSITY,74,30000.0,26018.0,26018.0,905.0,905.0,0.0,0.0,0.0,0.0,23872.0,23872.0,23872.0,2,49,403,Private,86.0,AMERICAN UNIVERSITY,AMERICAN,37,American University,DC,100000.0,56000.0,AMERICAN UNIVERSITY
4,APPALACHIAN SCHOOL OF LAW,60,9475.0,17500.0,17500.0,500.0,500.0,0.0,0.0,0.0,0.0,20100.0,20100.0,20100.0,4,8,31,Private,92.0,APPALACHIAN SCHOOL OF LAW,APPALACHIAN,186,Appalachian School of Law,VA,NaN,NaN,APPALACHIAN SCHOOL OF LAW


## Calculations

In [33]:
# Add per year columns
final_df["tuition_year_nonres"] = 2*final_df["tuition_semester_nonres"]
final_df["tuition_year_res"] = 2*final_df["tuition_semester_res"]
final_df["fees_year_nonres"] = 2*final_df["fees_semester_nonres"]
final_df["fees_year_res"] = 2*final_df["fees_semester_res"]

## Upload Data

In [34]:
drop_cols = [
    "school_clean",
    "school_root",
    "No.",
    "Law School",
    "State",
    "school_match"
]

rename_cols = {
    "Median Salary Private":"med_salary_private",
    "Median Salary Public":"med_salary_public"
}

upload_df = final_df.drop(drop_cols, axis=1).rename(columns=rename_cols)

upload_df.head()

,school,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours,med_salary_private,med_salary_public,tuition_year_nonres,tuition_year_res,fees_year_nonres,fees_year_res
0,"AKRON, UNIVERSITY OF",75,7000.0,12107.0,12157.0,1419.0,1469.0,712.0,712.0,1419.0,1469.0,22404.0,22404.0,7038.0,0,8,129,Public,88.0,60000.0,42198.0,24314.0,24214.0,2938.0,2838.0
1,"ALABAMA, UNIVERSITY OF",86,24000.0,11805.0,21530.0,310.0,310.0,0.0,0.0,0.0,0.0,19010.0,19010.0,19010.0,1,7,133,Public,90.0,70000.0,50000.0,43060.0,23610.0,620.0,620.0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,93,31000.0,26053.0,26053.0,135.0,135.0,0.0,0.0,0.0,0.0,0.0,16640.0,7690.0,1,11,136,Private,87.0,73000.0,59000.0,52106.0,52106.0,270.0,270.0
3,AMERICAN UNIVERSITY,74,30000.0,26018.0,26018.0,905.0,905.0,0.0,0.0,0.0,0.0,23872.0,23872.0,23872.0,2,49,403,Private,86.0,100000.0,56000.0,52036.0,52036.0,1810.0,1810.0
4,APPALACHIAN SCHOOL OF LAW,60,9475.0,17500.0,17500.0,500.0,500.0,0.0,0.0,0.0,0.0,20100.0,20100.0,20100.0,4,8,31,Private,92.0,NaN,NaN,35000.0,35000.0,1000.0,1000.0


In [39]:
upload_df.to_sql(
    "school_stats", 
    db_engine, 
    schema="debt_calc_user", 
    if_exists="replace", 
    index=False
)

In [36]:
query = """
    SELECT *
    FROM debt_calc_user.school_stats
"""

pd.read_sql(query, db_engine).set_index("school").head()

,percent_get_grant,median_grant,tuition_semester_res,tuition_semester_nonres,fees_semester_res,fees_semester_nonres,tuition_credit_res,tuition_credit_nonres,fees_credit_res,fees_credit_nonres,col_on_campus,col_off_campus,col_at_home,unemployed_not_seeking,unemployed_seeking,total_graduates,school_type,grad_credit_hours,med_salary_private,med_salary_public,tuition_year_nonres,tuition_year_res,fees_year_nonres,fees_year_res
school,,,,,,,,,,,,,,,,,,,,,,,,
"AKRON, UNIVERSITY OF",75,7000.0,12107.0,12157.0,1419.0,1469.0,712.0,712.0,1419.0,1469.0,22404.0,22404.0,7038.0,0,8,129,Public,88.0,60000.0,42198.0,24314.0,24214.0,2938.0,2838.0
"ALABAMA, UNIVERSITY OF",86,24000.0,11805.0,21530.0,310.0,310.0,0.0,0.0,0.0,0.0,19010.0,19010.0,19010.0,1,7,133,Public,90.0,70000.0,50000.0,43060.0,23610.0,620.0,620.0
ALBANY LAW SCHOOL OF UNION UNIVERSITY,93,31000.0,26053.0,26053.0,135.0,135.0,0.0,0.0,0.0,0.0,0.0,16640.0,7690.0,1,11,136,Private,87.0,73000.0,59000.0,52106.0,52106.0,270.0,270.0
AMERICAN UNIVERSITY,74,30000.0,26018.0,26018.0,905.0,905.0,0.0,0.0,0.0,0.0,23872.0,23872.0,23872.0,2,49,403,Private,86.0,100000.0,56000.0,52036.0,52036.0,1810.0,1810.0
APPALACHIAN SCHOOL OF LAW,60,9475.0,17500.0,17500.0,500.0,500.0,0.0,0.0,0.0,0.0,20100.0,20100.0,20100.0,4,8,31,Private,92.0,NaN,NaN,35000.0,35000.0,1000.0,1000.0
